# Imports

In [1]:
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
import pandas as pd
from datetime import datetime as DT
import xarray as xr

In [3]:
from imod import msw
from imod import mf6
import primod

In [4]:
import flopy as fp

In [5]:
import importlib as IL
IL.reload(U)
IL.reload(UIM)
IL.reload(primod)
IL.reload(msw)
IL.reload(mf6)

<module 'imod.mf6' from 'c:\\Users\\Karam014\\OneDrive - Universiteit Utrecht\\WS_Mdl\\code\\.pixi\\envs\\default\\Lib\\site-packages\\imod\\mf6\\__init__.py'>

In [ ]:
# Additional imports for shapefile creation
from shapely.geometry import LineString, Point
import geopandas as gpd
import re

# Options + Basics

In [7]:
MdlN = 'NBr30'

In [8]:
# Load paths and variables from PRJ & INI
d_Pa = U.get_MdlN_Pa(MdlN)
Pa_PRJ = d_Pa['PRJ']
Dir_PRJ = PDN(Pa_PRJ)
d_INI = U.INI_to_d(d_Pa['INI'])
Xmin, Ymin, Xmax, Ymax = [float(i) for i in d_INI['WINDOW'].split(',')]
SP_date_1st, SP_date_last = [DT.strftime(DT.strptime(d_INI[f'{i}'], '%Y%m%d'), '%Y-%m-%d') for i in ['SDATE', 'EDATE']]
dx = dy = float(d_INI['CELLSIZE'])

🟢 - INI file C:/OD/WS_Mdl\models/NBr\code/Mdl_Prep/Mdl_Prep_NBr30.ini read successfully. Dictionary created with 22 keys.


In [9]:
d_Pa

{'Mdl': 'NBr',
 'MdlN': 'NBr30',
 'Pa_Mdl': 'C:/OD/WS_Mdl\\models/NBr',
 'Smk_temp': 'C:/OD/WS_Mdl\\models/NBr\\code/snakemake/temp',
 'PoP': 'C:/OD/WS_Mdl\\models/NBr\\PoP',
 'code': 'C:/OD/WS_Mdl\\code',
 'pixi': 'C:/OD/WS_Mdl\\pixi.toml',
 'coupler_Exe': 'C:/OD/WS_Mdl\\software/iMOD5/IMC_2024.4/imodc.exe',
 'MF6_DLL': 'C:/OD/WS_Mdl\\software/iMOD5/IMC_2024.4\\./modflow6/libmf6.dll',
 'MSW_DLL': 'C:/OD/WS_Mdl\\software/iMOD5/IMC_2024.4\\./metaswap/MetaSWAP.dll',
 'INI': 'C:/OD/WS_Mdl\\models/NBr\\code/Mdl_Prep/Mdl_Prep_NBr30.ini',
 'BAT': 'C:/OD/WS_Mdl\\models/NBr\\code/Mdl_Prep/Mdl_Prep_NBr30.bat',
 'PRJ': 'C:/OD/WS_Mdl\\models/NBr\\In/PRJ/NBr30.prj',
 'Smk': 'C:/OD/WS_Mdl\\models/NBr\\code/snakemake/NBr30.smk',
 'Sim': 'C:/OD/WS_Mdl\\models/NBr\\Sim',
 'Pa_MdlN': 'C:/OD/WS_Mdl\\models/NBr\\Sim/NBr30',
 'TOML': 'C:/OD/WS_Mdl\\models/NBr\\Sim/NBr30\\imod_coupler.toml',
 'TOML_iMOD5': 'C:/OD/WS_Mdl\\models/NBr\\Sim/NBr30\\NBr30.toml',
 'LST_Sim': 'C:/OD/WS_Mdl\\models/NBr\\Sim/NBr30\\

# Read SFR

## Read file

In [10]:
Pa_SFR = d_Pa['SFR']

In [11]:
l_Lns = U.r_Txt_Lns(Pa_SFR)

## PACKAGEDATA 

### Read into DF

In [49]:
PkgDt_start = next(i for i,l in enumerate(l_Lns) if 'BEGIN PACKAGEDATA' in l)+2
PkgDt_end = next(i for i,l in enumerate(l_Lns) if 'END PACKAGEDATA' in l)
PkgDt_Cols = ['ifno', 'L', 'R', 'C', 'rlen', 'rwid', 'rgrd', 'rtp', 'rbth', 'rhk', 'man', 'ncon', 'ustrf', 'ndv', 'aux', 'X', 'Y']
PkgDt_data = [l.split() for l in l_Lns[PkgDt_start:PkgDt_end] if l.strip() and not l.strip().startswith('#')]

for row in PkgDt_data: # Robust fix: if only one 'NONE' and it's at index 1, replace with three 'NONE's
    if row.count('NONE') == 1 and row[1] == 'NONE':
        row[1:2] = ['NONE','NONE','NONE']

DF_PkgDt = gpd.GeoDataFrame(PkgDt_data, columns=PkgDt_Cols)
DF_PkgDt

,ifno,L,R,C,rlen,rwid,rgrd,rtp,rbth,rhk,man,ncon,ustrf,ndv,aux,X,Y
0,1,1,1,6,25,2,0.001,4.1240,0.001,0.0001,0.03,1,1,0,r1,113237.5,396187.5
1,2,1,1,11,25,2,0.001,1.7740,0.001,0.0001,0.03,1,1,0,r2,113362.5,396187.5
2,3,1,1,12,25,2,0.001,1.7790,0.001,0.0001,0.03,1,1,0,r3,113387.5,396187.5
3,4,1,1,18,25,2,0.001,0.5950,0.001,0.0001,0.03,1,1,0,r4,113537.5,396187.5
4,5,1,1,20,25,2,0.001,0.2070,0.001,0.0001,0.03,1,1,0,r5,113587.5,396187.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38375,38376,1,344,343,25,2,0.001,19.4960,0.001,0.0001,0.03,1,1,0,r38376,121662.5,387612.5
38376,38377,1,344,346,25,2,0.001,19.6470,0.001,0.0001,0.03,1,1,0,r38377,121737.5,387612.5
38377,38378,1,344,349,25,2,0.001,19.9650,0.001,0.0001,0.03,1,1,0,r38378,121812.5,387612.5
38378,38379,1,344,433,25,2,0.001,23.0440,0.001,0.0001,0.03,1,1,0,r38379,123912.5,387612.5


### Cleanup

In [13]:
DF_PkgDt = DF_PkgDt.replace(["NONE", "", "NaN", "nan"], pd.NA) # 1) normalize NA-like tokens and strip spaces
DF_PkgDt = DF_PkgDt.apply(lambda s: s.str.strip() if s.dtype == "object" else s)

l_Num_Cols = [c for c in DF_PkgDt.columns if c != 'aux']  # 2) choose numeric columns and coerce
DF_PkgDt[l_Num_Cols] = DF_PkgDt[l_Num_Cols].apply(pd.to_numeric)

# 3) optional: get nullable ints/floats
DF_PkgDt = DF_PkgDt.convert_dtypes()
# or enforce ints: DF_PkgDt[num_cols] = DF_PkgDt[num_cols].astype("Int64")


In [14]:
DF_PkgDt.dtypes

ifno              Int64
L                 Int64
R                 Int64
C                 Int64
rlen              Int64
rwid              Int64
rgrd            Float64
rtp             Float64
rbth            Float64
rhk             Float64
man             Float64
ncon              Int64
ustrf             Int64
ndv               Int64
aux      string[python]
X               Float64
Y               Float64
dtype: object

## CONNECTIONDATA

In [15]:
Conn_start = next(i for i,l in enumerate(l_Lns) if 'BEGIN CONNECTIONDATA' in l)+1
Conn_end = next(i for i,l in enumerate(l_Lns) if 'END CONNECTIONDATA' in l)
Conn_data = [(int(parts[0]), [int(x) for x in parts[1:]])
             for l in l_Lns[Conn_start+1:Conn_end]
             if (parts := l.strip().split())]

DF_Conn = pd.DataFrame(Conn_data, columns=['reach_N','connections'])
DF_Conn['downstream'] = DF_Conn['connections'].apply(lambda l_Conns: next((-x for x in l_Conns if x < 0), None))
DF_Conn['downstream'] = DF_Conn['downstream'].astype("Int64")
DF_Conn

,reach_N,connections,downstream
0,1,[-7],7
1,2,[-7],7
2,3,[-7],7
3,4,[-7],7
4,5,[-7],7
...,...,...,...
38375,38376,[-7],7
38376,38377,[-7],7
38377,38378,[-7],7
38378,38379,[-7],7


## Merge

In [16]:
DF = pd.merge(DF_PkgDt, DF_Conn[['reach_N', 'downstream']], left_on='ifno', right_on='reach_N', how='left')

In [17]:
DF.insert(0, 'reach_N', DF.pop('reach_N'))

In [20]:
DF.drop('ifno', axis=1, inplace=True)

In [22]:
DF = pd.merge(DF, DF[['reach_N', 'X', 'Y']].rename(columns={'reach_N':'downstream', 'X':'DStr_X', 'Y':'DStr_Y'}), on='downstream', how='left')

In [ ]:
def make_geom(row, DF):
    if pd.notna(row['downstream']):
        # Get downstream X,Y
        DS = DF.loc[DF['ifno'] == row['downstream']]
        if not DS.empty:
            return LineString([(row['X'], row['Y']), (DS.iloc[0]['X'], DS.iloc[0]['Y'])])
    # If no downstream, return Point (or buffer for circle)
    return Point(row['X'], row['Y']).buffer(dx)

GDF = DF.copy()
GDF['geometry'] = GDF.apply(lambda r: make_geom(r, GDF), axis=1)
GDF = gpd.GeoDataFrame(GDF, geometry='geometry', crs=28992)  # Set CRS as needed
GDF.head()

In [23]:
GDF = DF.copy()

In [27]:
GDF['geometry'] = GDF.apply(lambda row: LineString([(row['X'], row['Y']), (row['DStr_X'], row['DStr_Y'])]) if pd.notnull(row['DStr_X']) and pd.notnull(row['DStr_Y']) else Point(row['X'], row['Y']).buffer(dx), axis=1)

In [28]:
GDF = gpd.GeoDataFrame(GDF, geometry='geometry', crs=28992)  # Set CRS as needed

In [29]:
GDF.head(10)

,reach_N,L,R,C,rlen,rwid,rgrd,rtp,rbth,rhk,...,ncon,ustrf,ndv,aux,X,Y,downstream,DStr_X,DStr_Y,geometry
0,1,1,1,6,25,2,0.001,4.124,0.001,0.0001,...,1,1,0,r1,113237.5,396187.5,7,113637.5,396187.5,"LINESTRING (113237.5 396187.5, 113637.5 396187.5)"
1,2,1,1,11,25,2,0.001,1.774,0.001,0.0001,...,1,1,0,r2,113362.5,396187.5,7,113637.5,396187.5,"LINESTRING (113362.5 396187.5, 113637.5 396187.5)"
2,3,1,1,12,25,2,0.001,1.779,0.001,0.0001,...,1,1,0,r3,113387.5,396187.5,7,113637.5,396187.5,"LINESTRING (113387.5 396187.5, 113637.5 396187.5)"
3,4,1,1,18,25,2,0.001,0.595,0.001,0.0001,...,1,1,0,r4,113537.5,396187.5,7,113637.5,396187.5,"LINESTRING (113537.5 396187.5, 113637.5 396187.5)"
4,5,1,1,20,25,2,0.001,0.207,0.001,0.0001,...,1,1,0,r5,113587.5,396187.5,7,113637.5,396187.5,"LINESTRING (113587.5 396187.5, 113637.5 396187.5)"
5,6,1,1,21,25,2,0.001,0.064,0.001,0.0001,...,1,1,0,r6,113612.5,396187.5,7,113637.5,396187.5,"LINESTRING (113612.5 396187.5, 113637.5 396187.5)"
6,7,1,1,22,25,2,0.001,-0.08,0.001,0.0001,...,18121,1,0,r7,113637.5,396187.5,38380,123912.5,387612.5,"LINESTRING (113637.5 396187.5, 123912.5 387612.5)"
7,8,1,1,26,25,2,0.001,0.635,0.001,0.0001,...,1,1,0,r8,113737.5,396187.5,7,113637.5,396187.5,"LINESTRING (113737.5 396187.5, 113637.5 396187.5)"
8,9,1,1,53,25,2,0.001,5.429,0.001,0.0001,...,1,1,0,r9,114412.5,396187.5,7,113637.5,396187.5,"LINESTRING (114412.5 396187.5, 113637.5 396187.5)"
9,10,1,1,54,25,2,0.001,5.404,0.001,0.0001,...,1,1,0,r10,114437.5,396187.5,7,113637.5,396187.5,"LINESTRING (114437.5 396187.5, 113637.5 396187.5)"


In [44]:
Pa_SHP = PJ(d_Pa['PoP'], f'In/SFR/SFR_{MdlN}.gpkg')

In [45]:
Pa_SHP

'C:/OD/WS_Mdl\\models/NBr\\PoP\\In/SFR/SFR_NBr30.gpkg'

In [46]:
os.makedirs(PDN(Pa_SHP), exist_ok=True)

In [48]:
GDF.to_file(Pa_SHP, driver='GPKG')

# Test out funtion

In [51]:
from WS_Mdl import geo as G

In [53]:
IL.reload(G)

<module 'WS_Mdl.geo' from 'C:\\Users\\Karam014\\OneDrive - Universiteit Utrecht\\WS_Mdl\\code\\WS_Mdl\\geo.py'>

In [54]:
G.SFR_to_GPKG(MdlN)

🟢 - INI file C:/OD/WS_Mdl\models/NBr\code/Mdl_Prep/Mdl_Prep_NBr30.ini read successfully. Dictionary created with 22 keys.
🟢 - SFR for NBr30 has been converted to GPKG and saved at C:/OD/WS_Mdl\models/NBr\PoP\In/SFR/SFR_NBr30.gpkg.
